In [ ]:
from PIL import Image, ImageOps                   
import numpy as np                     
import os
import matplotlib.pyplot as plt      
plt.style.use("ggplot")

In [ ]:
def img_data(imgPath,disp = True):
    
    orig_img = Image.open(imgPath)
    
    img_size_kb = os.stat(imgPath).st_size/1024
    
    ori_pixels = np.array(orig_img.getdata()).reshape(*orig_img.size, -1)
    
    img_dim = ori_pixels.shape 
    
    if disp:
        plt.imshow(orig_img)
        plt.show()
    
    data_dict = {}
    data_dict['img_size_kb'] = img_size_kb
    data_dict['img_dim'] = img_dim
    
    return data_dict

In [ ]:
imgPath = 'test1.jpg';

data_dict_ori = img_data(imgPath)

print('Original Image Data')
print('Original Image size (kB)',data_dict_ori['img_size_kb'])
print('Original Image Shape',data_dict_ori['img_dim'])

In [ ]:
from sklearn.decomposition import PCA

def pca_compose(imgPath):
    
    orig_img = Image.open(imgPath)
    img = np.array(orig_img.getdata())
    img = img.reshape(*orig_img.size, -1)
    pca_channel = {}
    img_t = np.transpose(img) 
    
    for i in range(img.shape[-1]):    
        
        per_channel = img_t[i] 
        
        channel = img_t[i].reshape(*img.shape[:-1])  
        
        pca = PCA(random_state = 42)                
        
        fit_pca = pca.fit_transform(channel)        
        
        pca_channel[i] = (pca,fit_pca)  
        
    return pca_channel

In [ ]:
pca_channel = pca_compose(imgPath)
len(pca_channel)

In [ ]:
var_exp = explained_var_n(pca_channel, 1023)

print("Explained variance in percentage by PCA : ", var_exp*100,"%")

In [ ]:
def variance_added_pc(pca_channel):
    
    var_exp_channel = [];var_exp=0;
    
    for channel in pca_channel:
        pca,_ = pca_channel[channel]
        var_exp_channel.append(pca.explained_variance_ratio_)
        
    var_exp = (var_exp_channel[0]+var_exp_channel[1]+var_exp_channel[2])/3
    
    x = list(var_exp);y = list(range(1,1+len(x)));y = list(range(1,21))
    
    plt.yticks(np.arange(0, max(x)+0.05,0.05))
    plt.xticks(np.arange(min(y), max(y)+1,1))
    plt.title("Individual Variance for each Principal Component")
    plt.ylabel('Variance')
    plt.xlabel('Principal Component');
    plt.bar(y,x[:20],color = 'black')
    plt.show()

In [ ]:
variance_added_pc(pca_channel)

In [ ]:
def plot_variance_pc(pca_channel):
    
    pca,fit_pca = pca_channel[0]
    
    exp_var = {}
    
    for i in range(len(pca.components_)):
        var_exp = explained_var_n(pca_channel,i)
        exp_var[i+1] = var_exp
    
    lists = sorted(exp_var.items()) 
    
    x, y = zip(*lists) 
    
    pt90 = next(xx[0] for xx in enumerate(y) if xx[1] > 0.9)
    pt95 = next(xx[0] for xx in enumerate(y) if xx[1] > 0.95)
    
    plt.plot(x, y)
    plt.vlines(x=x[pt90], ymin=0, ymax=y[pt90], colors='green',  ls=':', lw=2, label=str('90% Variance Explained : n = '+str(x[pt90])))
    plt.vlines(x=x[pt95], ymin=0, ymax=y[pt95], colors='red', ls=':', lw=2, label=str('95% Variance Explained : n = '+str(x[pt95])))
    
    plt.xticks(np.arange(min(x)-1, max(x)-1,100))
    plt.yticks(np.arange(0, max(y),0.1))
    
    plt.legend(loc="lower right")
    plt.title("Variance vs Principal Components")
    plt.xlabel("Principal Components")
    plt.ylabel("Variance Explained")
    plt.grid(True)
    plt.show()

In [ ]:
plot_variance_pc(pca_channel)